# Differential Privacy for Vision Tasks

## Settings und Imports

In [ ]:
# suppress warnings
import warnings

warnings.filterwarnings('ignore')

#autoreload other packages when code changed
%load_ext autoreload
%autoreload 2

In [ ]:
import torch

torch.manual_seed(20)  #Reproduzierbarkeit
from torch import nn
from torch.utils.data import DataLoader
import torchvision

import opacus
from opacus import PrivacyEngine
from opacus.validators import ModuleValidator
from opacus.utils.batch_memory_manager import BatchMemoryManager

from tqdm.notebook import tqdm
import gc
import os

In [ ]:
#Own Code
from privacyflow.configs import path_configs
from privacyflow.datasets import faces_dataset
from privacyflow.models import face_models

In [ ]:
#Check if GPU is available
if torch.cuda.is_available():
    print("GPU will be used")
    device = torch.device('cuda')
else:
    print("No GPU available")
    device = torch.device('cpu')

## Data Prep

In [ ]:
label_columns = 'all'  #40 attributes

data_augmentation_train = torchvision.transforms.Compose([
    #torchvision.transforms.AutoAugment(),
    torchvision.transforms.ToTensor(),
])

data_augmentation_test = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor()
])

data_augmentation_train_with_resize = torchvision.transforms.Compose([
    torchvision.transforms.Resize((224, 224)),
    #torchvision.transforms.AutoAugment(),
    torchvision.transforms.ToTensor(),
])

data_augmentation_test_with_resize = torchvision.transforms.Compose([
    torchvision.transforms.Resize((224, 224)),
    torchvision.transforms.ToTensor()
])


train_dataset_cnn = faces_dataset.FacesDataset(label_cols=label_columns, mode="train",transform=data_augmentation_train)
val_dataset_cnn = faces_dataset.FacesDataset(label_cols=label_columns, mode="val", transform=data_augmentation_test)
test_dataset_cnn = faces_dataset.FacesDataset(label_cols=label_columns, mode="test", transform=data_augmentation_test)

train_dataset_vit = faces_dataset.FacesDataset(label_cols=label_columns, mode="train",transform=data_augmentation_train_with_resize)
val_dataset_vit = faces_dataset.FacesDataset(label_cols=label_columns, mode="val",transform=data_augmentation_test_with_resize)
test_dataset_vit = faces_dataset.FacesDataset(label_cols=label_columns, mode="test",transform=data_augmentation_test_with_resize)

## Models - No DP 

The following section contains code for training and testing the base models, which are trained without DPSGD

In [ ]:
def train_model(model:nn.Module,
                criterion:nn.Module,
                optimizer:torch.optim.Optimizer,
                epochs:int,
                train_ds:torch.utils.data.Dataset,
                val_ds:torch.utils.data.Dataset,
                batch_size:int =32,
                num_workers:int=0,
                amount_labels:int=40,
                val:bool=True):
    torch.cuda.empty_cache()
    train_dl = DataLoader(train_ds, batch_size=batch_size,shuffle=True,num_workers=num_workers)
    for epoch in range(epochs):
        model.train()
        epoch_loss = 0.0
        for model_inputs, labels in tqdm(train_dl, leave=False):
            model_inputs = model_inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            model_outputs = model(model_inputs)
            loss = criterion(model_outputs, labels)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
        print(f"Epoch: {epoch + 1:2}",
              f"Train Loss: {epoch_loss / len(train_dl):.5f}")

        if val:
            torch.cuda.empty_cache()
            gc.collect() #free cuda memory from train_dataloader
            val_dl = DataLoader(val_ds, batch_size=batch_size, shuffle=False,num_workers=num_workers)
            val_loss = 0.0
            num_corrects = 0
            model.eval()
            for model_inputs, labels in val_dl:
                model_inputs = model_inputs.to(device)
                labels = labels.to(device)
                model_outputs = model(model_inputs)
                loss = criterion(model_outputs, labels)
                val_loss += loss.item()
                num_corrects += int((model_outputs.round() == labels).sum())
            print(f"Val Loss: {val_loss / len(val_dl):.5f}",
                  f"Val Accuracy (all attributes): {num_corrects / (len(val_ds) * amount_labels)}")

@torch.no_grad()
def test_model(model:nn.Module,
               test_ds:torch.utils.data.Dataset,
               batch_size:int,
               num_workers:int =0,
               amount_labels=40):
    test_dl = DataLoader(test_ds, batch_size=batch_size,num_workers=num_workers,shuffle=False)
    model.eval()
    num_corrects = 0
    for model_inputs, labels in tqdm(test_dl, leave=False):
        model_inputs = model_inputs.to(device)
        labels = labels.to(device)
        model_outputs = model(model_inputs)
        num_corrects += int((model_outputs.round() == labels).sum())
    print(f"Test Accuracy (all attributes): {num_corrects / (len(test_ds) * amount_labels)}")

In [ ]:
#ResNet
pretrained = False

model_cnn_base = face_models.get_FaceModelResNet(40, pretrained=pretrained).to(device)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model_cnn_base.parameters(), lr=0.01)

train_model(model=model_cnn_base,
            criterion=criterion,
            optimizer=optimizer,
            epochs=4,
            train_ds=train_dataset_cnn,
            val_ds=val_dataset_cnn,
            batch_size=128,
            num_workers=8,
            amount_labels=40)

test_model(model=model_cnn_base, test_ds=test_dataset_cnn, batch_size=128, num_workers=8)
torch.save(model_cnn_base.state_dict(), path_configs.FACE_CNN_MODEL)

In [ ]:
#Vision Transformer
pretrained = False

model_vit_base = face_models.get_FaceVisionTransformer(40, pretrained=pretrained).to(device)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model_vit_base.heads.parameters(), lr=0.01)

train_model(model=model_vit_base,
            criterion=criterion,
            optimizer=optimizer,
            epochs=10,
            train_ds=train_dataset_vit,
            val_ds=train_dataset_vit,
            batch_size=32,
            num_workers=8,
            amount_labels=40)

torch.save(model_vit_base.state_dict(), path_configs.FACE_VIT_MODEL)
test_model(model=model_vit_base, test_ds=test_dataset_vit, batch_size=64,num_workers=4)

## Model + DP-SGD

The following code contains code for training models with DPSGD. 
The ResNet-18 and the ViT Models iterates through lists of different param values and train a model for each of the param combination.
The model are stored and the testing is done in a seperate cell. 

The reason why training in testing are exceuted in different cells is because of the memory on the GPU. Having the training dataloader and the test dataloader in memory may cause a CUDA OOM.

In [ ]:
def train_model_dpsgd(model:nn.Module,
                criterion:nn.Module,
                optimizer: opacus.optimizers.optimizer.DPOptimizer,
                train_dl:torch.utils.data.DataLoader,
                privacy_engine:opacus.PrivacyEngine,
                max_physical_batch_size:int = 16,
                val_dl:torch.utils.data.DataLoader = None,
                len_val_ds:int = 1,
                epochs:int = 5,
                amount_labels:int=40,
                max_epsilon:int= 10,
                val:bool=True):
    epsilon_reached = False
    for epoch in range(epochs):
        if epsilon_reached:
            break
        print(f"Start Training Epoch: {epoch + 1:2}")
        model.train()
        epoch_loss = 0.0
        with (BatchMemoryManager(
                data_loader=train_dl,
                max_physical_batch_size=max_physical_batch_size, 
                optimizer=optimizer) 
        as memory_safe_data_loader):
            for model_inputs, labels in tqdm(memory_safe_data_loader,leave=False):
                model_inputs = model_inputs.to(device)
                labels = labels.to(device)
                #Forward + Backprop
                optimizer.zero_grad()
                model_outputs = model(model_inputs)
                loss = criterion(model_outputs, labels)
                loss.backward()
                optimizer.step()
                epoch_loss += loss.item()
                #Check if epsilon exceeds threshold after each batch
                if max_epsilon < privacy_engine.accountant.get_epsilon(delta=1e-6):
                    print(f"ε Value {max_epsilon:2} reached in Epoch {epoch+1:2}")
                    epsilon_reached = True
                    break

        print(f"Finished Training Epoch: {epoch + 1:2}",
              f"Train Loss: {epoch_loss / len(train_dl):.5f}",
              f"ε:{privacy_engine.accountant.get_epsilon(delta=1e-6):.5f}")

        if val:
            torch.cuda.empty_cache()
            gc.collect() #free cuda memory from train_dataloader
            val_loss = 0.0
            num_corrects = 0
            model.eval()
            for model_inputs, labels in val_dl:
                model_inputs = model_inputs.to(device)
                labels = labels.to(device)
                model_outputs = model(model_inputs)
                loss = criterion(model_outputs, labels)
                val_loss += loss.item()
                num_corrects += int((model_outputs.round() == labels).sum())
            print(f"Val Loss: {val_loss / len(val_dl):.5f}",
                  f"Val Accuracy (all attributes): {num_corrects / (len_val_ds * amount_labels)}")
        print("-------------------------------------------------------------")

In [ ]:
#ResNet-18 Parameters
cnn_clipping_threshold = 0.00001
cnn_pretrained = [True, False]
cnn_target_epsilon = [1,5,10]
cnn_num_epochs = [1,3,5,10]
cnn_batch_size = 256


#Dataloader for ResNet with DPSGD
data_augmentation_train = torchvision.transforms.Compose([
    #torchvision.transforms.AutoAugment(),
    torchvision.transforms.ToTensor(),
])

train_dataset_cnn = faces_dataset.FacesDataset(label_cols=label_columns, mode="train",transform=data_augmentation_train)
#The spezified Batch Sizes are Virtual Batch Sizes
train_dl_cnn_dpsgd = DataLoader(
    dataset=train_dataset_cnn,
    batch_size=cnn_batch_size
)

In [ ]:
#train different ResNet dpsgd models
for pretrained in cnn_pretrained:
    for target_epsilon in cnn_target_epsilon:
        for num_epochs in cnn_num_epochs:
            torch.cuda.empty_cache()
            gc.collect() #free cuda memory from train_dataloader
            model_cnn_dpsgd = face_models.get_FaceModelResNet(40,pretrained=pretrained).to(device)
            model_cnn_dpsgd = ModuleValidator.fix(model_cnn_dpsgd)
            criterion = nn.BCELoss()
            optimizer = torch.optim.Adam(model_cnn_dpsgd.parameters(), lr=0.01)
            privacy_engine= PrivacyEngine(accountant='rdp')
            model_cnn_dpsgd, optimizer, train_dl = privacy_engine.make_private_with_epsilon(
                module=model_cnn_dpsgd,
                optimizer=optimizer,
                data_loader=train_dl_cnn_dpsgd,
                epochs=num_epochs,
                target_epsilon=target_epsilon,
                target_delta=1e-6,
                max_grad_norm=cnn_clipping_threshold #Gradienten größer als dieser Wert werden geclippt
            )
            print(f"Training ResNet Model\npretrained={pretrained}\nNum Epochs = {num_epochs}\ntarget_epsilon={target_epsilon}\nNoise Mult={optimizer.noise_multiplier:.4f}")
            torch.cuda.empty_cache()
            gc.collect() #free cuda memory from train_dataloader
            train_model_dpsgd(model=model_cnn_dpsgd,
                              criterion=criterion,
                              optimizer=optimizer,
                              train_dl=train_dl,
                              privacy_engine=privacy_engine,
                              max_physical_batch_size=64,
                              max_epsilon=target_epsilon,
                              epochs=num_epochs,
                              val=False)
            torch.save(model_cnn_dpsgd._module.state_dict(), f"{path_configs.MODELS_TRAINED_BASE_PATH}/cnn_{'pretrained_' if pretrained else ''}epsilon{target_epsilon}_epochs{num_epochs}_clipp{str(cnn_clipping_threshold).replace('.','')}_batch256_ohneAA.pl" )

In [ ]:
#eval dpsgd models
for pretrained in cnn_pretrained:
    for epsilon in cnn_target_epsilon:
        for clipping_threshold in [cnn_clipping_threshold]:
            for num_epochs in cnn_num_epochs:
                model_path = f"{path_configs.MODELS_TRAINED_BASE_PATH}/cnn_{'pretrained_' if pretrained else ''}epsilon{epsilon}_epochs{num_epochs}_clipp{str(clipping_threshold).replace('.','')}_batch{cnn_batch_size}.pl"
                if not os.path.isfile(model_path):
                    continue
                #Load DPSGD Models
                model_dgsgd_testing = face_models.get_FaceModelResNet(40)
                model_dgsgd_testing = ModuleValidator.fix(model_dgsgd_testing)
                model_dgsgd_testing.load_state_dict(torch.load(model_path))
                model_dgsgd_testing = model_dgsgd_testing.to(device)
        
        
                print(f"Testing ResNet DPSGD Model with params:\npretrained={pretrained}\nNum Epochs = {num_epochs}\nepsilon={epsilon}\nclip={clipping_threshold}")
                test_model(model_dgsgd_testing,test_dataset_cnn,batch_size=64,num_workers=8)
                print("-------------------------------------")

In [ ]:
#ViT Parameters
vit_clipping_threshold = 0.00001
vit_pretrained = [True, False]
vit_target_epsilon = [1,5,10]
vit_num_epochs = [1,3,5]
vit_batch_size = 128

#Dataloader for ViT with DPSGD
data_augmentation_train_with_resize = torchvision.transforms.Compose([
    torchvision.transforms.Resize((224, 224)),
    #torchvision.transforms.AutoAugment(),
    torchvision.transforms.ToTensor()
])

train_dataset_vit = faces_dataset.FacesDataset(label_cols=label_columns, mode="train",transform=data_augmentation_train_with_resize)
#The spezified Batch Sizes are Virtual Batch Sizes
train_dl_vit_dpsgd = DataLoader(
    dataset=train_dataset_vit,
    batch_size=vit_batch_size
)

In [ ]:
#train different Vit dpsgd models
for pretrained in vit_pretrained:
    for target_epsilon in vit_target_epsilon:
        for num_epochs in vit_num_epochs:
            torch.cuda.empty_cache()
            gc.collect() #free cuda memory from train_dataloader
            model_vit_dpsgd = face_models.get_FaceVisionTransformer(40,pretrained=pretrained).to(device)
            model_vit_dpsgd = ModuleValidator.fix(model_vit_dpsgd)
            criterion = nn.BCELoss()
            optimizer = torch.optim.Adam(model_vit_dpsgd.parameters(), lr=0.01)
            privacy_engine= PrivacyEngine(accountant="rdp")
            model_vit_dpsgd, optimizer, train_dl = privacy_engine.make_private_with_epsilon(
                module=model_vit_dpsgd,
                optimizer=optimizer,
                data_loader=train_dl_vit_dpsgd,
                epochs=num_epochs,
                target_epsilon=target_epsilon,
                target_delta=1e-6,
                max_grad_norm=vit_clipping_threshold #Gradienten größer als dieser Wert werden geclippt
            )
            print(f"Training ViT Model\npretrained={pretrained}\nNum Epochs = {num_epochs}\ntarget_epsilon={target_epsilon}\n")
            torch.cuda.empty_cache()
            gc.collect() #free cuda memory from train_dataloader
            train_model_dpsgd(model=model_vit_dpsgd,
                              criterion=criterion,
                              optimizer=optimizer,
                              train_dl=train_dl,
                              privacy_engine=privacy_engine,
                              max_physical_batch_size=16,
                              max_epsilon=target_epsilon,
                              epochs=num_epochs,
                              val=False)
            torch.save(model_vit_dpsgd._module.state_dict(), f"{path_configs.MODELS_TRAINED_BASE_PATH}/vit_{'pretrained_' if pretrained else ''}epsilon{target_epsilon}_epochs{num_epochs}_clip{str(vit_clipping_threshold).replace('.','')}_batch{vit_batch_size}.pl" )

In [ ]:
# eval vit dpsgd models
for pretrained in vit_pretrained:
    for epsilon in vit_target_epsilon:
        for clipping_threshold in [vit_clipping_threshold]:
            for num_epochs in vit_num_epochs:
                model_path = f"{path_configs.MODELS_TRAINED_BASE_PATH}/vit_{'pretrained_' if pretrained else ''}epsilon{epsilon}_epochs{num_epochs}_clip{str(clipping_threshold).replace('.','')}_batch{vit_batch_size}.pl"
                if not os.path.isfile(model_path):
                    continue
                # Load DPSGD Models
                model_vit_testing = face_models.get_FaceVisionTransformer(40)
                model_vit_testing = ModuleValidator.fix(model_vit_testing)
                model_vit_testing.load_state_dict(torch.load(model_path))
                model_vit_testing = model_vit_testing.to(device)

                print(f"Testing ViT DPSGD Model with params:\npretrained={pretrained}\nNum Epochs = {num_epochs}\nepsilon={epsilon}\nclip={str(clipping_threshold).replace('.','')}")
                test_model(model_vit_testing,test_dataset_vit,batch_size=32,num_workers=4)
                print("-------------------------------------")